In [1]:
from pyspark.sql import SparkSession
import warnings

import pandas as pd
from pyspark.sql.functions import max, min, avg, year, dayofyear, to_date, date_format, dayofweek, hour, month, length
from pyspark.context import SparkContext as sc
from pyspark.sql.types import StructField, StringType, StructType
from pyspark.sql import SQLContext

warnings.filterwarnings('ignore')

spark=SparkSession.builder.master("local[*]").appName("mach_bda_dataviz").config("spark.driver.memory", "16g").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
sqlContext = SQLContext(spark)

#file to load is churn.csv in the same directory

dataset = spark.read.option("header", "true").option("inferSchema", "true").csv("churn.csv")
dataset.show(5)

your 131072x1 screen size is bogus. expect trouble
24/06/06 20:02:14 WARN Utils: Your hostname, DESKTOP-R8N4U8E resolves to a loopback address: 127.0.1.1; using 172.23.28.41 instead (on interface eth0)
24/06/06 20:02:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/06 20:02:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+------+-----------+---------------+-------------------+------------+-----------------------+-----------+---------------------+-------------------+---------------+-------------------+---------------------+--------------+---------------------+------------------------+----------------+---------------------+----------------------------+--------------+-------------------+--------------------+----------------+
|age|gender|security_no|region_category|membership_category|joining_date|joined_through_referral|referral_id|preferred_offer_types|medium_of_operation|internet_option|    last_visit_time|days_since_last_login|avg_time_spent|avg_transaction_value|avg_frequency_login_days|points_in_wallet|used_special_discount|offer_application_preference|past_complaint|   complaint_status|            feedback|churn_risk_score|
+---+------+-----------+---------------+-------------------+------------+-----------------------+-----------+---------------------+-------------------+---------------+-------

In [2]:
#remove negative valies from column 'days_since_last_login'

dataset = dataset.filter(dataset['days_since_last_login'] > 0)
dataset.show(5)

+---+------+-----------+---------------+-------------------+------------+-----------------------+-----------+---------------------+-------------------+---------------+-------------------+---------------------+--------------+---------------------+------------------------+----------------+---------------------+----------------------------+--------------+-------------------+--------------------+----------------+
|age|gender|security_no|region_category|membership_category|joining_date|joined_through_referral|referral_id|preferred_offer_types|medium_of_operation|internet_option|    last_visit_time|days_since_last_login|avg_time_spent|avg_transaction_value|avg_frequency_login_days|points_in_wallet|used_special_discount|offer_application_preference|past_complaint|   complaint_status|            feedback|churn_risk_score|
+---+------+-----------+---------------+-------------------+------------+-----------------------+-----------+---------------------+-------------------+---------------+-------

In [3]:
#remove rows with 'security_no' string length greater than 7
#remove rows with 'security_no' string length less than 1


dataset = dataset.filter(length(dataset['security_no']) <= 7)

dataset.count()

34990

In [4]:
#remove rows with avg_time_spent less than 0

dataset = dataset.filter(dataset['avg_time_spent'] >= 0)
dataset.count()

33355

In [5]:
#remove rows with avg_frequency_login_days set to Error

dataset = dataset.filter(dataset['avg_frequency_login_days'] != "Error")
dataset.count()


30188

In [6]:
#remove rows with points_in_wallet less or equal to 0

dataset = dataset.filter(dataset['points_in_wallet'] > 0)
dataset.count()

27253

In [7]:
#remove rows with joined_through_referral set to ?

dataset = dataset.filter(dataset['joined_through_referral'] != "?")
dataset.count()

#save the cleaned dataset to a new csv file

dataset.write.csv("dataCleaned.csv", header=True)

23256